In [17]:
import pandas as pd
from datetime import datetime, timedelta
import time

In [2]:
path = 'E:/Study/data/drop.csv'

In [7]:
columns = ['n_transfer', 'start_time', 'bus', 'subway', 'rer', 'tramway', 't_trip', 't_sep','day_start','day_end']

In [14]:
def get_counts(anything):
    counts = {}
    for a in anything:
        if a in counts:
            counts[a] = counts[a] + 1
        else:
            counts[a] = 1
    return counts

In [8]:
def change_Ot_time(data):
    return timedelta(hours=int(data['Ot'][6:9]) , minutes = int(data['Ot'][10:12]), seconds = int(data['Ot'][13:15]))
def change_Dt_time(data):
    if pd.isnull(data['Dt']):
        return timedelta(seconds = 0)
    else:
        return timedelta(hours=int(data['Dt'][6:9]) , minutes = int(data['Dt'][10:12]), seconds = int(data['Dt'][13:15]))

In [9]:
def get_time_sep(data):
    day_start = data.iloc[0]['Ot'].seconds
    day_end = data.iloc[len(data)-1]['Dt'].seconds
    rer = 0
    bus = 0
    subway = 0
    tramway = 0
    n_transfer = 1
    t_trip = 0
    trip_id = 0
    results = pd.DataFrame(columns=columns)
    t_end = data.iloc[0]['Dt'].seconds
    t_start = data.iloc[0]['Ot'].seconds
    for i in range(1,len(data)):
        mode = data.iloc[i-1]['O_mode']
        if mode == 1:
            rer = rer + 1
        elif mode == 2:
            bus = bus + 1
        elif mode == 3:
            subway = subway + 1
        else:
            tramway = tramway + 1
        t_first = data.iloc[i-1]['Dt'].seconds
        t_second = data.iloc[i]['Ot'].seconds
        sep_time = t_second - t_first
        if sep_time<3600:
            t_end = data.iloc[i]['Dt'].seconds
            n_transfer = n_transfer + 1
        else:
            results.loc[trip_id] = [n_transfer, t_start, bus, subway, rer, tramway, (t_end - t_start), sep_time,day_start,day_end]
            t_start = data.iloc[i]['Ot'].seconds
            t_end = data.iloc[i]['Dt'].seconds
            rer = 0
            bus = 0
            subway = 0
            tramway = 0
            n_transfer =1
            trip_id = trip_id + 1
    return results

In [3]:
results = pd.read_csv(path, header=0)

In [13]:
results['Ot'] = results.apply(change_Ot_time, axis=1)
results['Dt'] = results.apply(change_Dt_time, axis=1)

In [15]:
id_s = [a[0] for a in get_counts(results['client_id']).items()]

In [22]:
len(id_s)

217678

In [24]:
m_results = pd.DataFrame()
test = id_s[200000:]
for a in test:
    data = results[results['client_id'] == a]
    m_results = m_results.append(get_time_sep(data), ignore_index =True)
m_results.to_csv('E:/Study/data/final_data_5.csv', index=False)  